In [2]:
import sys
import os
from dotenv import load_dotenv

load_dotenv('config/.env')

# Add the src directory to the system path
sys.path.append(os.path.abspath('../src'))

openai_api_key = os.getenv('OPENAI_API_KEY')
tavily_api_key = os.getenv('TAVILY_API_KEY')

In [3]:
import openai
openai.api_key = openai_api_key



In [ ]:
import helper_fns as hp
from langgraph.adaptive_rag import (
    QueryAnalyzer,
    QueryTransformer,
    Document,
    FusionRetrievalRAG,
    Reranker,
    HallucinationGrader,
    AnswerGrader,
    WebRetriever,
    create_langgraph_app,
);

/Users/cboyfly/Documents/repos/hybrid-retrieval/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/cboyfly/Documents/repos/hybrid-retrieval/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /Users/cboyfly/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
12/16/2024 22:30:28 - INFO - 	 missing_keys: []
12/16/2024 22:30:28 - INFO - 	 unexpected_keys: []
12/16/2024 22:30:28 - INFO - 	 mismatched_keys: []
12/16/2024 22:30:28 - INFO - 	 error_msgs: []
12/16/2024 22:30:28 - INFO - 	 Model Parameters: 590.0M, Transformer: 434.6M, Coref head: 155.4M


In [4]:
text = hp.process_file('../data/building-blocks-of-rag-ebook-final.pdf')

12/06/2024 08:08:15 - INFO - 	 Tokenize 1 inputs...
Map: 100%|██████████| 1/1 [00:00<00:00,  5.18 examples/s]
12/06/2024 08:08:15 - INFO - 	 ***** Running Inference on 1 texts *****
Inference:   0%|          | 0/1 [00:00<?, ?it/s]

IndexError occurred: list index out of range. Returning original text.


# Classes Demo

## Query Analyzer

In [ ]:
knowledge_base_description = "Insider trading"
analyzer = QueryAnalyzer(knowledge_base_description=knowledge_base_description)

In [7]:
response = analyzer.analyze_query(question="Which teams did Michael Jordan play for?")
print(response)

12/06/2024 08:08:29 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


no-retrieval


In [12]:
response = analyzer.analyze_query(question="How can quantum computing improve AI development in the near future?")
print(response)

12/04/2024 21:08:50 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


vectorstore


## Query Transformations

In [6]:
rephrased_question = QueryTransformer().rewrite_question(question="Why should I care about quantum computing?")
stepback_question = QueryTransformer().generate_stepback_query(question="Why should I care about quantum computing?")
sub_questions = QueryTransformer().decompose_question(question="Why should I care about quantum computing?")

12/06/2024 08:13:34 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12/06/2024 08:13:38 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12/06/2024 08:13:40 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [7]:
print(f"Rephrased question: {rephrased_question} \nStep-back question: {stepback_question} \nSub-questions: {sub_questions}")

Rephrased question: What are the key reasons to be interested in quantum computing and its potential impact on technology and society? 
Step-back question: What are the implications and significance of emerging technologies like quantum computing? 
Sub-questions: ['What are the fundamental principles of quantum computing?', 'What are the potential applications of quantum computing in various fields?', 'How does quantum computing differ from classical computing?', 'What are the current advancements and challenges in quantum computing research?']


## Answer Grader

In [6]:
question = "What is the difference between legal and illegal insider trading?"
answer = "Legal insider trading involves buying or selling stock by company insiders who follow disclosure regulations and do not use material, nonpublic information. Illegal insider trading involves trading based on material, nonpublic information, violating a duty of trust or confidence."
graded_answer = AnswerGrader().grade_answer(question=question,generation=answer)
print(graded_answer)

12/06/2024 19:13:30 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


yes


In [7]:
question = "What are some recent high-profile cases of insider trading?"
answer = "Legal insider trading involves buying or selling stock by company insiders who follow disclosure regulations and do not use material, nonpublic information. Illegal insider trading involves trading based on material, nonpublic information, violating a duty of trust or confidence."
graded_answer = AnswerGrader().grade_answer(question=question,generation=answer)
print(graded_answer)

12/06/2024 19:15:13 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


no


## Halucination Grader

In [ ]:
context = """
Insider trading is the buying or selling of a publicly traded company's stock or other securities based on material, nonpublic information about the company. This information is typically obtained through a position of trust or access within the company, such as by executives, employees, or other insiders like lawyers, accountants, or consultants.

Insider trading can be legal or illegal, depending on the circumstances:

Legal Insider Trading: Occurs when company insiders, such as executives or board members, buy or sell stock in their own company but do so in compliance with regulations. They must disclose their trades to regulatory bodies (e.g., the SEC in the U.S.) and avoid trading based on nonpublic, material information.

Illegal Insider Trading: Occurs when someone trades securities based on material, nonpublic information in violation of their duty of trust or confidence. This includes tipping off others who then trade on the information. Illegal insider trading undermines market fairness and investor confidence.

Key regulators, like the SEC, enforce laws to prevent and prosecute illegal insider trading. Penalties can include fines, disgorgement of profits, and even imprisonment.
"""

answer = "Legal insider trading involves buying or selling stock by company insiders who follow disclosure regulations and do not use material, nonpublic information. Illegal insider trading involves trading based on material, nonpublic information, violating a duty of trust or confidence."

hallucination_grade = HallucinationGrader().grade_hallucination(documents=context, generation=answer)
print(hallucination_grade)

12/06/2024 19:36:53 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


yes


## Document Reranker

In [4]:
docs = [
    Document(page_content=(
        "Insider trading is the buying or selling of a publicly traded company's stock or other securities "
        "based on material, nonpublic information about the company. This information is typically obtained "
        "through a position of trust or access within the company, such as by executives, employees, or other insiders."
    )),
    Document(page_content=(
        "Legal insider trading occurs when company insiders, such as executives or board members, buy or sell stock "
        "in their own company while following regulations. They must disclose their trades to regulatory bodies, "
        "such as the SEC, and avoid trading based on nonpublic, material information."
    )),
    Document(page_content=(
        "Illegal insider trading occurs when someone trades securities based on material, nonpublic information "
        "in violation of their duty of trust or confidence. This can include tipping off others who trade on the information."
    )),
    Document(page_content=(
        "Regulators like the SEC enforce laws to prevent and prosecute illegal insider trading. Penalties for violations "
        "include fines, disgorgement of profits, and imprisonment, which help maintain market fairness and investor confidence."
    )),
    Document(page_content=(
        "Financial markets thrive on transparency and trust. Effective regulatory oversight ensures fair access to information "
        "for all investors, fostering confidence in the system."
    )
    ),
]

reranker = Reranker()
query = "Explain insider trading"

top_docs = reranker.rerank_documents(query=query, documents=docs, top_n=3)

for doc_info in top_docs:
    print(doc_info['relevance_score'], doc_info['content'].page_content)

12/07/2024 09:29:26 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12/07/2024 09:29:26 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12/07/2024 09:29:27 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12/07/2024 09:29:28 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12/07/2024 09:29:28 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


10 Insider trading is the buying or selling of a publicly traded company's stock or other securities based on material, nonpublic information about the company. This information is typically obtained through a position of trust or access within the company, such as by executives, employees, or other insiders.
9 Legal insider trading occurs when company insiders, such as executives or board members, buy or sell stock in their own company while following regulations. They must disclose their trades to regulatory bodies, such as the SEC, and avoid trading based on nonpublic, material information.
9 Illegal insider trading occurs when someone trades securities based on material, nonpublic information in violation of their duty of trust or confidence. This can include tipping off others who trade on the information.


In [4]:
question = "Do LLMs have feelings?"
websearch = WebRetriever(api_key=tavily_api_key)
retrieved_docs = websearch.retrieve(query=question)

print("Retrieved Documents:")
for idx, doc in enumerate(retrieved_docs):
    print(f"\nDocument {idx+1}: {doc.page_content}")

Retrieved Documents:

Document 1: Can LLMs have Emotions "I believe that LLMs can still connect with humans on an emotional level. After all, humans are also capable of understanding and responding to emotions that are expressed through text. For example, if an LLM writes a text that expresses sadness, a human can still understand and empathize with that sadness, even though

Document 2: To what degree do they feel the specific qualia that we associate with emotions? ... I made this point and that is why I did not try to argue that LLMs did not have qualia. But I do believe you can consider necessary conditions and look at their absence. For instance, I can safely declare that a rock does not have qualia, because I know it does

Document 3: The answer depends on the definition of ‘feelings.’ According to the American Psychological Association’s, autoregressive Large Language Models (LLMs) trained and fully prompted with text tokens cannot: Their ‘state of mind’ is trivial (a singleton)

# Langgraph App Demo

In [ ]:
context = """
Insider trading is the buying or selling of a publicly traded company's stock or other securities based on material, nonpublic information about the company. This information is typically obtained through a position of trust or access within the company, such as by executives, employees, or other insiders like lawyers, accountants, or consultants.

Insider trading can be legal or illegal, depending on the circumstances:

Legal Insider Trading: Occurs when company insiders, such as executives or board members, buy or sell stock in their own company but do so in compliance with regulations. They must disclose their trades to regulatory bodies (e.g., the SEC in the U.S.) and avoid trading based on nonpublic, material information.

Illegal Insider Trading: Occurs when someone trades securities based on material, nonpublic information in violation of their duty of trust or confidence. This includes tipping off others who then trade on the information. Illegal insider trading undermines market fairness and investor confidence.

Key regulators, like the SEC, enforce laws to prevent and prosecute illegal insider trading. Penalties can include fines, disgorgement of profits, and even imprisonment.
"""
question = "When is insider trading legal?"
docs = FusionRetrievalRAG(text=context).run(query=question)
print(docs)

12/13/2024 11:54:30 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
12/13/2024 11:54:31 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
12/13/2024 11:54:31 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


["Insider trading is the buying or selling of a publicly traded company's stock or other securities based on material, nonpublic information about the company. This information is typically obtained through a position of trust or access within the company, such as by executives, employees, or other insiders like lawyers, accountants, or consultants.\n\nInsider trading can be legal or illegal, depending on the circumstances:\n\nLegal Insider Trading: Occurs when company insiders, such as executives or board members, buy or sell stock in their own company but do so in compliance with regulations. They must disclose their trades to regulatory bodies (e.g., the SEC in the U.S.) and avoid trading based on nonpublic, material information.", 'Illegal Insider Trading: Occurs when someone trades securities based on material, nonpublic information in violation of their duty of trust or confidence. This includes tipping off others who then trade on the information. Illegal insider trading undermi

In [4]:
context = """
Insider trading is the buying or selling of a publicly traded company's stock or other securities based on material, nonpublic information about the company. This information is typically obtained through a position of trust or access within the company, such as by executives, employees, or other insiders like lawyers, accountants, or consultants.

Insider trading can be legal or illegal, depending on the circumstances:

Legal Insider Trading: Occurs when company insiders, such as executives or board members, buy or sell stock in their own company but do so in compliance with regulations. They must disclose their trades to regulatory bodies (e.g., the SEC in the U.S.) and avoid trading based on nonpublic, material information.

Illegal Insider Trading: Occurs when someone trades securities based on material, nonpublic information in violation of their duty of trust or confidence. This includes tipping off others who then trade on the information. Illegal insider trading undermines market fairness and investor confidence.

Key regulators, like the SEC, enforce laws to prevent and prosecute illegal insider trading. Penalties can include fines, disgorgement of profits, and even imprisonment.
"""
kb_description = "legal vs illegal insider trading"
app = create_langgraph_app(knowledge_base_description=kb_description, 
                           text=context)

In [5]:
from pprint import pprint

question = "When is insider trading legal?"
# Run
inputs = {
    "question": question
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---


12/14/2024 20:14:32 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12/14/2024 20:14:32 - INFO - 	 Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


---ROUTE QUESTION TO RAG---
---RETRIEVE---


12/14/2024 20:14:33 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
12/14/2024 20:14:33 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
12/14/2024 20:14:33 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---


...

---CHECK HALLUCINATIONS---


12/14/2024 20:14:39 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---


12/14/2024 20:14:40 - INFO - 	 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
('Insider trading is generally illegal when it involves buying or selling '
 'securities based on material, nonpublic information. However, it can be '
 "legal if company insiders, like executives, trade their own company's stock "
 'while complying with regulations, including timely disclosure of their '
 'trades to regulatory bodies. Different countries have varying regulations '
 'governing insider trading, and penalties for illegal insider trading can '
 'include fines and imprisonment.')
